In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from tqdm import tqdm

In [4]:
os.getcwd()

'C:\\Users\\USER\\PycharmProjects\\ERC'

경로설정

In [66]:
directory= 'C:\\Users\\USER\\PycharmProjects\\ERC\\'
os.chdir(directory)

annotation 취합
---

In [67]:
ann = os.path.join(directory, 'data/20/annotation')
files = os.listdir(ann)
filescnt_wav = [i for i in files if i.endswith('.csv')]

df_all = pd.DataFrame()
for i in range(0, len(filescnt_wav)) :
    file = filescnt_wav[i]
    df = pd.read_csv(os.path.join(ann, file), encoding = 'utf-8')
    df['filename'] = file.split('_eval')[0]
    df_all = pd.concat([df_all, df])

sess_df = df_all.drop([0],axis = 0)
sess_df.head()

,Numb,WAV,,Segment ID,Total Evaluation,.1,.2,Eval01F,.3,.4,...,Eval08M,.17,.18,Eval09F,.19,.20,Eval10M,.21,.22,filename
1,1,3.5783901,11.637391,Sess01_script01_User002M_001,neutral,3.4,2.9,neutral,3,3,...,neutral,4,3,neutral,3,3,neutral,3,3,Sess01
2,2,11.637391,23.334393,Sess01_script01_User002M_002,neutral,3.1,2.9,neutral,3,3,...,neutral,4,3,neutral,3,3,neutral,3,3,Sess01
3,3,23.334393,31.558392,Sess01_script01_User002M_003,neutral,3.1,3,neutral,3,3,...,neutral,4,3,neutral,3,3,neutral,3,3,Sess01
4,4,31.558392,42.97039,Sess01_script01_User002M_004,neutral,3.7,3.1,neutral,4,3,...,neutral,4,4,neutral,3,3,neutral,4,3,Sess01
5,5,42.77729,44.86329,Sess01_script01_User001F_001,neutral,3.8,2.8,happy,5,2,...,neutral,4,3,neutral,3,3,happy,4,3,Sess01


wav/txt 취합
---

In [68]:
def search(dirname):
    result = []
    filenames = os.listdir(dirname)
    for filename in filenames:
        full_filename = dirname + '//' + filename
        result.append(full_filename)    
    return result

In [69]:
wav_path = os.path.join(directory, 'data/20/wav')
wav_sess = search(wav_path)

In [70]:
txt_files = []
wav_files = []

for i in wav_sess:
    filenames = os.listdir(i)
    direc = i[42:]
    for filename in filenames:
        full_filename = direc + '//' + filename
        if full_filename[-3:] == 'wav':
            wav_files.append(full_filename)
        elif full_filename[-3:] == 'txt':
            txt_files.append(full_filename)
        else:
            print('wrong!')

In [71]:
total = pd.DataFrame({'Audio': wav_files,
                      'Text': txt_files,
                      'Emotion': wav_files,
                      'Arousal': wav_files,
                      'Valence': wav_files})

전체 데이터 취합
---

In [72]:
path = directory+'data/20/'
for i in tqdm(range(total.shape[0])):
    directory = os.path.join(path, total.iloc[i, 1])
    with open (directory, 'r') as text:
        string = text.readlines()
    total.iloc[i, 1] = string[0].replace('\n', '')

100%|██████████████████████████████████████████████████████████████████████████| 13462/13462 [00:02<00:00, 4818.59it/s]


In [73]:
for i in tqdm(range(total.shape[0])):
    file_name = total.iloc[i, 2][16:-4]
    total.iloc[i, 2] = sess_df[sess_df['Segment ID'] == file_name].iloc[0, 4]
    total.iloc[i, 3] = sess_df[sess_df['Segment ID'] == file_name].iloc[0, 6]
    total.iloc[i, 4] = sess_df[sess_df['Segment ID'] == file_name].iloc[0, 5]

total.head()

100%|███████████████████████████████████████████████████████████████████████████| 13462/13462 [00:34<00:00, 394.50it/s]


,Audio,Text,Emotion,Arousal,Valence
0,wav//Session01//Sess01_script01_User001F_001.wav,n/ 아 친구들도? l/,neutral,2.8,3.8
1,wav//Session01//Sess01_script01_User001F_002.wav,l/ 나는 생일?,neutral,3.1,3.3
2,wav//Session01//Sess01_script01_User001F_003.wav,생일날이면은 b/ 내가 고기를 되게 좋아하니까 엄마도 그걸 아니까 미역국도 같이 해...,neutral,3.1,3.3
3,wav//Session01//Sess01_script01_User001F_004.wav,b/ 선물이라 이 보통 돈으로 주시지. l/,happy,3.1,3.6
4,wav//Session01//Sess01_script01_User001F_005.wav,l/ 용돈으로 주시고.,neutral,3.2,3.5


In [74]:
for i in range(total.shape[0]):
    text = total.iloc[i, 1]
    text = text.replace('c/','').replace('n/','').replace('N/','').replace('u/','').replace('l/','')
    text = text.replace('b/','').replace('*','').replace('+','').replace(',','')
    total.iloc[i, 1] = text

In [75]:
total.to_csv('C:\\Users\\USER\\PycharmProjects\\ERC\\data\\total.csv', index=False, encoding="utf-8-sig") 
total.head()

,Audio,Text,Emotion,Arousal,Valence
0,wav//Session01//Sess01_script01_User001F_001.wav,아 친구들도?,neutral,2.8,3.8
1,wav//Session01//Sess01_script01_User001F_002.wav,나는 생일?,neutral,3.1,3.3
2,wav//Session01//Sess01_script01_User001F_003.wav,생일날이면은 내가 고기를 되게 좋아하니까 엄마도 그걸 아니까 미역국도 같이 해주시...,neutral,3.1,3.3
3,wav//Session01//Sess01_script01_User001F_004.wav,선물이라 이 보통 돈으로 주시지.,happy,3.1,3.6
4,wav//Session01//Sess01_script01_User001F_005.wav,용돈으로 주시고.,neutral,3.2,3.5


In [ ]:
import os
from tqdm import tqdm
import torch
import librosa
import torchaudio
import numpy as np
import pandas as pd

def Mel_Extractor(file_name, length):
    signal, sr = torchaudio.load(file_name)
    mel_spec = librosa.power_to_db(
        librosa.feature.melspectrogram(y=signal[0].numpy(),
                                       sr=sr, n_mels=128, fmax=8000, fmin=100), ref=np.max)
    mel_delta = librosa.feature.delta(mel_spec)

    mel_result = np.stack((mel_spec.transpose(), mel_delta.transpose()), axis=0)
    mel_result = torch.from_numpy(mel_result)

    if mel_result.shape[1] > length:
        mel_result = mel_result[:, 0:length, :]
    else:
        mel_result = torch.cat([mel_result, torch.zeros(mel_result.shape[0], length - mel_result.shape[1], mel_result.shape[2])], dim=1)
        
    mel_result = mel_result.numpy()

    return mel_result

for i in tqdm(range(total.shape[0])):
    file_name = os.path.join(path, total.iloc[i, 0])
    audio_feature = Mel_Extractor(file_name=file_name, length=512)
    total.iloc[i, 0] = audio_feature

In [ ]:
import gzip
import pickle

In [ ]:
with gzip.open(path + 'data/Data_Original.pickle', 'wb') as f:
    pickle.dump(total, f)